## Creating a single file from the transcripts. Creating a meta file.

In [2]:
import os
import pickle
import pandas as pd
import requests

In [ ]:
directory = "../data/combined_transcripts_freeConv"

docs = []
meta = {}
i = 0
for filename in os.listdir(directory):
    with open(os.path.join(directory, filename), 'r', encoding="UTF-8") as f:
        txt = f.read()

    mordor = [e.strip() for e in txt.split("\n") if e.startswith("Mordor: ")]
    gondor = [e.strip() for e in txt.split("\n") if e.startswith("Gondor: ")]

    j = len(mordor)
    k = len(gondor)
    for h in range(j):
        meta[i+h] = filename + "_mordor_" + str(h)
    for g in range(k):
        meta[i+g+j] = filename + "_gondor_" + str(g)
    mordor = [e.replace("Mordor: ", "") for e in mordor]
    gondor = [e.replace("Gondor: ", "") for e in gondor]
    docs += mordor + gondor
    a = (j+k)
    i += a
with open("../resources/docs.pkl", "wb") as outfile:
    pickle.dump(docs, outfile)

with open("../resources/meta.pkl", "wb") as outfile:
    pickle.dump(meta, outfile)

print("Az adatbázis",len(docs), "beszédfordulót tartalmaz.")
print("Az adatbázis első beszédfordulójának metaadata:", meta[0])

In [ ]:
# make txt file from pickle file

In [ ]:
with open("../resources/docs.pkl", "rb") as f:
    docs = pickle.load(f)
with open("../resources/conversation.txt", "w", encoding='UTF-8') as f:
    for doc in docs:
        f.write(doc + "\n")

In [ ]:
# separate the speech turns with "csiribiri" and make it lower case. Save it as txt file.

In [ ]:
with open("../resources/conversation.txt", "r", encoding='UTF-8') as f:
    conversation = f.read()
conversation_2 = conversation.split("\n")
conversation_3 = conversation_2[:-1]
conversation_4 = "\ncsiribir\n".join(conversation_3)
conversation_4 = conversation_4.lower()
with open("../resources/conversation_csiribiri_lower.txt", "w", encoding='UTF-8') as f:
    f.write(conversation_4)

## Lemmatization and POS taging

In [ ]:
#docker run --rm -p5000:5000 -it -e "EMTSV_NUM_PROCESSES=4" mtaril/emtsv

In [ ]:
# Make the POST request
text = requests.post('http://127.0.0.1:5000/tok/morph/pos',
                  files={'file': open("../resources/conversation_csiribiri_lower.txt", encoding='UTF-8')})
# save file
with open("../results/conversation_lower_emtsv.txt", "w", encoding='UTF-8') as f:
    f.write(text.text)

In [3]:
# open txt file and make csv file from it
with open("../results/conversation_lower_emtsv.txt", "r", encoding='UTF-8') as f:
    conversation = f.read()
conversation_2 = conversation.split("\n")
df = pd.DataFrame([x.split("\t") for x in conversation_2], columns=["form", "wsafter", "anas", "lemma", "xpostag"])
df_2 = df.copy().drop(["wsafter","anas"], axis=1)

In [5]:
# lementjük csv-ben a 3 oszlopot (form, lemma, POS-tag) tartalmazó dataframe-et 
df_2.to_csv("../results/conversation_lower_emtsv.csv", index=False, encoding='UTF-8')

In [8]:
# készítünk egy nyers lemmatizált változatot, amely tartalmazza az írásjeleket, a hümmögést, hezitálást, nevetést
corpus_emtsv = pd.read_csv("../results/conversation_lower_emtsv.csv", encoding='UTF-8')
corpus_emtsv_2 = corpus_emtsv.dropna()
lemmatized_raw_1 = corpus_emtsv_2["lemma"].tolist()
lemmatized_raw_1.pop(0)
lemmatized_raw_2 = " ".join(lemmatized_raw_1)
lemmatized_raw_3 = lemmatized_raw_2.split("csiribir")
lemmatized_raw_4 = [x.split() for x in lemmatized_raw_3]

In [9]:
# lementjük a lemmatizált szöveget txt-ben és pkl-ben, amely tartalmazza az írásjeleket, a hümmögést, hezitálást, nevetést
with open("../results/lemmatized_raw.txt", "w") as outfile:
    for doc in lemmatized_raw_4:
        outfile.write(" ".join(doc) + "\n")
with open ("../results/lemmatized_raw.pkl", "wb") as f:
    pickle.dump(lemmatized_raw_4, f)

In [10]:
# készítünk egy lemmatizált változatot, amely nem tartalmazza az írásjeleket, a hümmögést, hezitálást, nevetést
corpus_no_punctuation = corpus_emtsv_2[corpus_emtsv_2["xpostag"] != "[Punct]"]
corpus_no_hum = corpus_no_punctuation[corpus_no_punctuation["lemma"] != "hum"]
corpus_no_hes = corpus_no_hum[corpus_no_hum["lemma"] != "hes"]
corpus_no_laugh = corpus_no_hes[corpus_no_hes["lemma"] != "Laugh"]
corpus_no_laugh_2 = corpus_no_laugh.dropna()
lemmatized_clean = corpus_no_laugh_2["lemma"].tolist()
lemmatized_clean.pop(0)
lemmatized_clean_2 = " ".join(lemmatized_clean)
lemmatized_clean_3 = lemmatized_clean_2.split("csiribir")
lemmatized_clean_4 = [x.split() for x in lemmatized_clean_3]

In [12]:
# lementjük a tiszta lemmatizált szöveget txt-ben és pkl-ben, amely nem tartalmazza az írásjeleket, a hümmögést, hezitálást, nevetést
with open("../results/lemmatized_clean.txt", "w") as outfile:
    for doc in lemmatized_clean_4:
        outfile.write(" ".join(doc) + "\n")
with open ("../results/lemmatized_clean.pkl", "wb") as f:
    pickle.dump(lemmatized_clean_4, f)

In [21]:
# kiszedjük a POS-tageket csv-ben és pkl-ben, amely nem tartalmazza az írásjeleket, a hümmögést, hezitálást, nevetést. A POS-tagelő nem tudja, hogy a "fog" és a "lesz" jövő idő, ezért átírjuk őket.
pos_table = corpus_no_laugh.copy()
pos_table.loc[(pos_table["lemma"] == "lesz") & (pos_table["xpostag"].str.contains("Prs")), "xpostag"] = pos_table["xpostag"].str.replace("Prs", "Fut")
pos_table.loc[(pos_table["lemma"] == "fog") & (pos_table["xpostag"].str.contains("Prs")), "xpostag"] = pos_table["xpostag"].str.replace("Prs", "Fut")
pos_table = pos_table.drop(pos_table.index[0])

In [22]:
# lementjük csak a POS tageket pickle-ben és csv-ben, a teljesen tiszta változatot, amelyben nincsen írásjel, hümmögés, hezitálás, nevetés
pos_table.to_csv("../results/postags_clean.csv", index=False, encoding='UTF-8')
pos_table.loc[pos_table["form"] == "csiribir", "xpostag"] = "csiribir"
pos_clean_1 = pos_table["xpostag"].tolist()
pos_clean_2 = " ".join(pos_clean_1)
pos_clean_3 = pos_clean_2.split("csiribir")
pos_clean_4 = [x.split() for x in pos_clean_3]
with open ("../results/postags_clean.pkl", "wb") as f:
    pickle.dump(pos_clean_4, f)

## Mondat tokenizálás

In [ ]:
from nltk.tokenize import PunktTokenizer

with open("../data/conversation.txt", "r", encoding='UTF-8') as f:
    conv_1 = f.read()
conv_2 = conv_1.split("\n")
conv_2.pop()
tokenizer = PunktTokenizer()
tokenized = [tokenizer.tokenize(line) for line in conv_2]
pickle.dump(tokenized, open("../results/tokenized_by_sentence.pkl", "wb"))